In [17]:
import numpy as np
import pandas as pd
import pickle

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import Sequential
from keras.utils import to_categorical
from keras.layers import Dense,SimpleRNN,Embedding,Flatten

In [8]:
train_ds = pd.read_csv('https://raw.githubusercontent.com/dinulimann/sentimentele/main/train.csv',encoding='latin1');
validation_ds = pd.read_csv('https://raw.githubusercontent.com/dinulimann/sentimentele/main/test.csv',encoding='latin1');

In [9]:
train_ds = train_ds[['text','sentiment']]
validation_ds = validation_ds[['text','sentiment']]

In [10]:
train_ds['text'].fillna('',inplace=True)
validation_ds['text'].fillna('',inplace=True)

<ipython-input-10-cb58bb3e5260>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_ds['text'].fillna('',inplace=True)
<ipython-input-10-cb58bb3e5260>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

In [11]:
def func(sentiment):
    if sentiment =='positive':
        return 0;
    elif sentiment =='negative':
        return 1;
    else:
        return 2;
train_ds['sentiment'] = train_ds['sentiment'].apply(func)
validation_ds['sentiment'] = validation_ds['sentiment'].apply(func)

In [12]:
x_train = np.array(train_ds['text'].tolist())
y_train = np.array(train_ds['sentiment'].tolist())
x_test = np.array(validation_ds['text'].tolist())
y_test = np.array(validation_ds['sentiment'].tolist())

In [13]:
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [14]:
tokenizer = Tokenizer(num_words=20000)

In [15]:
tokenizer.fit_on_texts(x_train)
tokenizer.fit_on_texts(x_test)

In [18]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
from google.colab import files
files.download('tokenizer.pickle')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [23]:
from keras.utils import pad_sequences
x_train = pad_sequences(x_train, padding='post', maxlen=35)  # Set maxlen to 35
x_test = pad_sequences(x_test, padding='post', maxlen=35)

In [24]:
x_train[0]

array([   1,  162,   19, 7713,   71,    1,  151,   49,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0], dtype=int32)

In [26]:
x_train.shape

(27481, 35)

In [27]:
model = Sequential()
model.add(Embedding(input_dim=20000, output_dim=5, input_length=35))
model.add(SimpleRNN(32,return_sequences=False))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.4256 - loss: 1.0636 - val_accuracy: 0.6928 - val_loss: 0.8057
Epoch 2/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.6483 - loss: 0.7908 - val_accuracy: 0.7568 - val_loss: 0.6317
Epoch 3/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.7575 - loss: 0.6088 - val_accuracy: 0.7622 - val_loss: 0.5839
Epoch 4/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.8185 - loss: 0.4810 - val_accuracy: 0.7545 - val_loss: 0.6175
Epoch 5/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8593 - loss: 0.3924 - val_accuracy: 0.7439 - val_loss: 0.6362
Epoch 6/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.8840 - loss: 0.3352 - val_accuracy: 0.7448 - val_loss: 0.6863
Epoch 7/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.9045 - loss: 0.2892 - val_accuracy: 0.7360 - val_loss: 0.7567
Epoch 8/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9146 - loss: 0.2579 - 

In [32]:
from google.colab import files
files.download('model.h5')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>